In [1]:
import numpy as np
import pandas as pd
import rebound
import mr_forecast as mr
import spock
import os
import glob
import random
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
% matplotlib inline

/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/xgboost/__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


In [2]:
plt.rc('lines', linewidth=6)
plt.rc('font', size=40, family='serif', weight=500)
plt.rc('mathtext', fontset = 'dejavuserif')
plt.rc('axes', linewidth=6, titlepad=20)
plt.rc('patch', linewidth=6)
plt.rc('xtick.major', width=5, size=20)
plt.rc('xtick.minor', width=4, size=15)
plt.rc('ytick.major', width=5, size=20)
plt.rc('ytick.minor', width=4, size=15)
plt.rc('figure.subplot', hspace=0.3, wspace=0.3)

In [3]:
def get_exit_times(dir_name):
    archive_list = glob.glob(dir_name + "/*.bin")
    
    exit_times = np.zeros(len(archive_list))
    periods = np.zeros(len(archive_list))
    identifiers = []
    
    for i, archive_filename in enumerate(archive_list):
        identifiers.append(archive_filename.split('system-')[1].split('.bin')[0])
        sa = rebound.SimulationArchive(archive_filename)
        
        first_sim = sa[0]
        periods[i] = first_sim.particles[1].P
        
        last_sim = sa[-1]
        exit_times[i] = last_sim.t
    
    return identifiers, periods, exit_times

def get_spock_probs(filename):
    data = np.load(filename)
    identifiers = data['identifier_list']
    probs = data['probs']
    
    return identifiers, probs

def get_multiplicities(identifiers):
    sysids = np.zeros(len(identifiers))
    places = np.zeros(len(identifiers))
    multiplicities = np.zeros(len(identifiers))
    
    for i, identifier in enumerate(identifiers):
        sysids[i] = int(identifier.rsplit('-')[0])
        places[i] = int(identifier.rsplit('-')[1])
        
    for sysid in sysids:
        multiplicities[sysids == sysid] +=1
    
    return multiplicities + 1

In [4]:
def get_results(sysid, place, dirname):
    filename = dirname + "spock-probs-system-" + str(sysid) + "-" + str(place) + ".npz"
    data = np.load(filename)
    
    return data

def get_files(dirname):
    file_list = glob.glob(dirname + "spock-probs-system-*")
    
    sys_list = []
    place_list = []
    
    for file in file_list:
        sys_list.append(int(file.split("-")[4]))
        place_list.append(int(file.split(".")[0].split("-")[5]))
    
    return sys_list, place_list

In [5]:
def get_edges_and_centres(min_val, max_val, N_bins=25):
    edges = np.linspace(min_val, max_val, N_bins+1)
    centres = (edges[1:] + edges[0:-1])/2
    
    return edges, centres

def get_summary_stats(x_vals, y_vals, z_vals, x_min, x_max, y_min, y_max, N_bins=25):
    x_edges, x_centres = get_edges_and_centres(x_min, x_max, N_bins)
    y_edges, y_centres = get_edges_and_centres(y_min, y_max, N_bins)
    
    x_grid, y_grid = np.meshgrid(x_centres, y_centres, indexing='ij')
    
    mean_grid = np.zeros((N_bins, N_bins))
    median_grid = np.zeros((N_bins, N_bins))
    std_grid = np.zeros((N_bins, N_bins))
    count_grid = np.zeros((N_bins, N_bins))
    
    for i in range(N_bins):
        x_index = np.logical_and(x_vals >= x_edges[i], x_vals < x_edges[i+1])
        for j in range(N_bins):
            y_index = np.logical_and(y_vals >= y_edges[j], y_vals < y_edges[j+1])
            joint_index = np.logical_and(x_index, y_index)
            
            mean_grid[i,j] = np.mean(z_vals[joint_index])
            median_grid[i,j] = np.median(z_vals[joint_index])
            std_grid[i,j] = np.std(z_vals[joint_index])
            count_grid[i,j] = np.sum(joint_index)
          
    return (x_grid, y_grid, mean_grid, median_grid, std_grid, count_grid)

In [13]:
def plot_prob_grid_mass_period(dirname, saveloc, sysid, place, pratio_lines=False, nbins=25):
    
    m_earth = 3e-6
    year = 2 * np.pi # one year in REBOUND time (in units where G=1)
    day = year / 365 # one day in REBOUND time (in units where G=1)
    
    with get_results(sysid, place, dirname) as data:
        mult = np.shape(data['period'])[0] - 1

        x = data['period'][-1] / day
        y = np.log10(data['mass'][-1] / m_earth)
        z = 1 - data['prob']
        
        m_1 = np.median(data['mass'][place]) / m_earth
        m_2 = np.median(data['mass'][place+1]) / m_earth
        P_1 = data['period'][place][0] / day
        P_2 = data['period'][place+1][0] / day
    
    scaled_period = (x - P_1) / (P_2 - P_1)
#     not_co_orbital = np.logical_and(scaled_period > 0.04, scaled_period < 0.96)
    not_co_orbital = scaled_period > 0
    
    prob_min = z[not_co_orbital].min()
    prob_msc = np.quantile(z[not_co_orbital], 0.05)
    prob_mean = z[not_co_orbital].mean()
    
#     fm13_dir = "../fang-margot-2013-comparison/simulation-archives/koi-gaia/integrated-archives/1e9-orbits/"
#     fm13_sa = rebound.SimulationArchive(fm13_dir + "system-" + str(sysid) + "-" + str(place) + ".bin")
#     fm13_mass = np.log10(3.671)
#     fm13_period = fm13_sa[0].particles[-1].P / day
#     fm13_prob = 1 - fm13_spock_probs[fm13_spock_identifiers == str(sysid) + "-" + str(place)][0]
    
    fig, ax = plt.subplots(2, 2, figsize=(30,30))

    im_viridis = []
    im_magma = []
    im_plasma = []

#     row_1_data = get_summary_stats(x, y, z, x.min(), x.max(), y.min(), y.max(), nbins)
    row_1_data = get_summary_stats(x, y, z, x.min(), x.max(), -1, 2, nbins)
    im_viridis.append(ax[0][0].pcolormesh(row_1_data[0], row_1_data[1], row_1_data[2], cmap='viridis_r', rasterized=True))
    im_viridis.append(ax[0][1].pcolormesh(row_1_data[0], row_1_data[1], row_1_data[3], cmap='viridis_r', rasterized=True))
    im_magma.append(ax[1][0].pcolormesh(row_1_data[0], row_1_data[1], row_1_data[4], cmap='magma', rasterized=True))
    im_plasma.append(ax[1][1].pcolormesh(row_1_data[0], row_1_data[1], row_1_data[5], cmap='plasma', rasterized=True))
    
#     ax[0][0].plot(fm13_period, fm13_mass, 'xw', markersize=40, markeredgewidth=5)
#     ax[0][1].plot(fm13_period, fm13_mass, 'xw', markersize=40, markeredgewidth=5)
#     ax[1][0].plot(fm13_period, fm13_mass, 'xw', markersize=40, markeredgewidth=5)
#     ax[1][1].plot(fm13_period, fm13_mass, 'xw', markersize=40, markeredgewidth=5)

    if pratio_lines:
        pratios = np.array([2, 3/2, 4/3, 5/4, 6/5])
        pratio_1 = pratios * P_1
        pratio_1 = pratio_1[pratio_1 < P_2]
        pratio_2 = P_2 / pratios
        pratio_2 = pratio_2[pratio_2 > P_1]
        
        for p in pratio_1:
            ax[0][0].axvline(p, ls='--', color='k')
            ax[0][1].axvline(p, ls='--', color='k')
        for p in pratio_2:
            ax[0][0].axvline(p, ls=':', color='k')
            ax[0][1].axvline(p, ls=':', color='k')
    
    norm_viridis = matplotlib.colors.Normalize(vmin=0, vmax=1)
    for im in im_viridis:
        im.set_norm(norm_viridis)

    vmin_magma = min(im.get_array().min() for im in im_magma)
    vmax_magma = max(im.get_array().max() for im in im_magma)
    norm_magma = matplotlib.colors.Normalize(vmin=vmin_magma, vmax=vmax_magma)
    for im in im_magma:
        im.set_norm(norm_magma)

    vmin_plasma = min(im.get_array().min() for im in im_plasma)
    vmax_plasma = max(im.get_array().max() for im in im_plasma)
    norm_plasma = matplotlib.colors.Normalize(vmin=vmin_plasma, vmax=vmax_plasma)
    for im in im_plasma:
        im.set_norm(norm_plasma)

    ax_viridis = inset_axes(ax[1][0], width="100%", height="100%", loc='center left', bbox_to_anchor=(-1, 1195, 90, 700))
    fig.colorbar(im_viridis[0], cax=ax_viridis, orientation="vertical")#, ticks=[1, 2, 3])
    ax_viridis.yaxis.set_ticks_position("left")
    ax_viridis.set_ylabel('Probability', labelpad=-250)

    ax_magma = inset_axes(ax[1][1], width="100%", height="100%", loc='center left', bbox_to_anchor=(-1, 275, 90, 700))
    fig.colorbar(im_magma[0], cax=ax_magma, orientation="vertical")
    ax_magma.yaxis.set_ticks_position("left")
    ax_magma.set_ylabel('Probability', labelpad=-250)

    ax_plasma = inset_axes(ax[1][1], width="100%", height="100%", loc='center right', bbox_to_anchor=(2100, 275, 90, 700))
    fig.colorbar(im_plasma[0], cax=ax_plasma, orientation="vertical")
    ax_plasma.set_ylabel('Number', labelpad=50, rotation=270)

    ax[0][0].set_xlabel('Period (days)')
    ax[0][1].set_xlabel('Period (days)')
    ax[1][0].set_xlabel('Period (days)')
    ax[1][1].set_xlabel('Period (days)')

    ax_row_list = np.array([0, 0, 1, 1])
    ax_col_list = np.array([0, 1, 0, 1])
    for i in range(4):
        ax[ax_row_list[i]][ax_col_list[i]].set_ylim(-1, 2)
        ax[ax_row_list[i]][ax_col_list[i]].set_yticks([-1, 0, 1, 2])
        ax[ax_row_list[i]][ax_col_list[i]].set_yticklabels(['$10^{-1}$','$10^{0}$','$10^{1}$','$10^{2}$'])
        ax[ax_row_list[i]][ax_col_list[i]].set_ylabel('Inserted planet mass ($M_{\oplus}$)')

    ax[0][0].set_title('Mean')
    ax[0][1].set_title('Median')
    ax[1][0].set_title('Standard Deviation')
    ax[1][1].set_title('Count')
    
#     ax[0][1].annotate("$N = ${mult:d}\n$M_1 = ${m_1:.1f}\n$M_2 = ${m_2:.1f}\n$p_{{FM13}} = ${fm13_prob:.4f}\n$p_{{min}} = ${prob_min:.4f}\n$p_{{5\%}} = ${prob_5:.4f}".format(\
#     mult=mult, m_1=m_1, m_2=m_2, fm13_prob=fm13_prob, prob_min=prob_min, prob_5=prob_5),
#             xy=(1, 0), xycoords='axes fraction',
#             xytext=(50, 450), textcoords='offset pixels',
#             horizontalalignment='left',
#             verticalalignment='top')

    ax[0][1].annotate("$N = ${mult:d}\n$M_{{in}} = ${m_1:.1f}$M_\oplus$\n$M_{{out}} = ${m_2:.1f}$M_\oplus$\n$p_{{min}} = ${prob_min:.4f}\n$p_{{MSC}} = ${prob_msc:.4f}\n$p_{{mean}} = ${prob_mean:.4f}".format(\
    mult=mult, m_1=m_1, m_2=m_2, prob_min=prob_min, prob_msc=prob_msc, prob_mean=prob_mean),
            xy=(1, 0), xycoords='axes fraction',
            xytext=(50, 450), textcoords='offset pixels',
            horizontalalignment='left',
            verticalalignment='top')
    
    plt.savefig("plots/" + saveloc + "system-" + str(sysid) + "-" + str(place) + ".pdf", bbox_inches='tight')
    plt.close()
    
    return None

In [7]:
df = pd.read_csv("cumulative_koi_gaia_bonomo.csv", comment="#")
sys_list, place_list = get_files("spock-probs/")

sys_list = np.array(sys_list)
place_list = np.array(place_list)

identifiers = []

for (sysid, place) in zip(sys_list, place_list):
    identifiers.append(str(sysid) + "-" + str(place))
    
identifiers = np.array(identifiers)

place_list = place_list[sys_list != 3245969]
sys_list = sys_list[sys_list != 3245969]

In [15]:
for i, (sysid, place) in enumerate(zip(sys_list[0:10], place_list[0:10])):
    plot_prob_grid_mass_period("spock-probs/sigma_e_p05_sigma_i_2p5/", "period-mass-grids/sigma_e_p05_sigma_i_2p5/", sysid, place, nbins=25)

/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [18]:
for i, (sysid, place) in enumerate(zip(sys_list, place_list)):
    plot_prob_grid_mass_period("spock-probs/sigma_e_p01_sigma_i_p5/", "period-mass-grids/sigma_e_p01_sigma_i_p5/", sysid, place, nbins=25)

/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [16]:
for i, (sysid, place) in enumerate(zip(sys_list, place_list)):
    plot_prob_grid_mass_period("spock-probs/sigma_e_p05_sigma_i_2p5/", "period-mass-grids/sigma_e_p05_sigma_i_2p5/", sysid, place, nbins=25)

/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/alysa/miniconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
